# NER Model

In [1]:
from datetime import datetime
import os

# import keras
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
# from keras.layers import Flatten, Dense, Embedding, Dropout, Bidirectional, LSTM, Concatenate, Reshape, Lambda, Input, Activation
# from keras.layers.merge import concatenate
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.text import one_hot
# from keras.utils import np_utils
# from keras_contrib.layers import CRF
# from model.data_utils import get_trimmed_glove_vectors, load_vocab, get_processing_word, CoNLLDataset, get_trimmed_glove_vectors, load_vocab, get_processing_word, minibatches, get_chunks, pad_sequences
# from model.ner_model import NERModel
from tensorflow.python.keras.layers import Flatten, Dense, Embedding, Dropout, Bidirectional, LSTM, Concatenate, Reshape, Lambda, Input, Activation, Masking
from tensorflow.python.keras.layers import concatenate
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import one_hot
from keras_contrib.layers import CRF
from model.data_utils import get_trimmed_glove_vectors, load_vocab, get_processing_word, CoNLLDataset, get_trimmed_glove_vectors, load_vocab, get_processing_word, minibatches, get_chunks, pad_sequences
from model.ner_model import NERModel

/home/mmyers/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#download new data
#source: https://github.com/synalp/NER
train_filename = "data/coNLL/eng/eng.train.iob"
dev_filename = "data/coNLL/eng/eng.testa.iob"
test_filename = "data/coNLL/eng/eng.testb.iob"

In [3]:
train_filename.count

<function str.count>

In [4]:
use_chars = True
max_iter = None

In [5]:
vocab_words = load_vocab("data/words.txt")
vocab_words

{'$NUM$': 13,
 '$UNK$': 9,
 '.': 16,
 'a': 11,
 'actor': 2,
 'american': 4,
 'an': 8,
 'and': 0,
 'economic': 12,
 'european': 22,
 'french': 7,
 'in': 5,
 'is': 1,
 'jean': 10,
 'lives': 21,
 'new': 19,
 'oscar': 20,
 'pierre': 14,
 'political': 17,
 'the': 3,
 'union': 6,
 'won': 18,
 'york': 15}

In [6]:
vocab_chars = load_vocab("data/chars.txt")
vocab_chars

{'.': 23,
 'A': 5,
 'E': 17,
 'F': 9,
 'J': 18,
 'N': 8,
 'P': 25,
 'T': 20,
 'U': 21,
 'Y': 3,
 'a': 15,
 'c': 27,
 'd': 11,
 'e': 7,
 'h': 24,
 'i': 10,
 'k': 16,
 'l': 1,
 'm': 26,
 'n': 12,
 'o': 14,
 'p': 22,
 'r': 4,
 's': 13,
 't': 6,
 'u': 19,
 'v': 2,
 'w': 0}

In [7]:
# NOTE: this order could be arbitrary, with values in the interval [0, num_tags]
# ALSO: there should be a difference between a null tag, and a padded label
vocab_tags = load_vocab("data/tags.txt")
# vocab_tags = { #maybe try a blank line or just 0 or set 0 equal to 0?
#         'O': 1,
#         'B-PER': 2,
#         'I-PER': 3,
#         'B-MISC': 4,
#         'I-MISC': 5,
#         'B-ORG': 6,
#         'I-ORG': 7,
#         'B-LOC': 8,
#         'I-LOC': 9
#     }
vocab_tags

{'B-LOC': 7,
 'B-MISC': 3,
 'B-ORG': 5,
 'B-PER': 1,
 'I-LOC': 8,
 'I-MISC': 4,
 'I-ORG': 6,
 'I-PER': 2,
 'O': 0}

In [8]:
n_words = len(vocab_words)

In [9]:
n_char = len(vocab_chars)
n_char

28

In [10]:
n_tags = (len(vocab_tags)+1) #+1 if different vocab_tags
n_tags

10

In [11]:
#coNLL data for validation
dev = CoNLLDataset(dev_filename, get_processing_word(vocab_words, vocab_chars,lowercase=True, chars=use_chars),
                  get_processing_word(vocab_tags, lowercase=False, allow_unk=False), max_iter)

In [12]:
# coNLL data for train
train = CoNLLDataset(train_filename, get_processing_word(vocab_words, vocab_chars,lowercase=True, chars=use_chars),
                  get_processing_word(vocab_tags, lowercase=False, allow_unk=False), max_iter)

In [13]:
# coNLL data for test
test = CoNLLDataset(test_filename, get_processing_word(vocab_words, vocab_chars,lowercase=True, chars=use_chars),
                  get_processing_word(vocab_tags, lowercase=False, allow_unk=False), max_iter)

In [14]:
dev

In [15]:
def get_glove_vocab(filename):
    """Load vocab from file
    Args:
        filename: path to the glove vectors
    Returns:
        vocab: set() of strings
    """
    print("Building vocab...")
    vocab = set()
    with open(filename) as f:
        for line in f:
            word = line.strip().split(' ')[0]
            vocab.add(word)
    print("- done. {} tokens".format(len(vocab)))
    return vocab

In [16]:
emb_data = np.load("data/glove.6B.300d.trimmed.npz")
# vocab_glove = get_glove_vocab("data/glove.6B.300d.trimmed.npz")
# vocab = vocab_words & vocab_glove

In [17]:
# emb_data = get_glove_vocab("data/glove.6B.300d.trimmed.npz")

In [18]:
embeddings = emb_data["embeddings"]
type(embeddings)

numpy.ndarray

In [19]:
dim_word = 300 #End to end paper uses 30
dim_char = 100

In [20]:
hidden_size_char = 100 # lstm on chars
hidden_size_lstm = 300 # lstm on word embeddings

In [21]:
lr = 0.0105 #0.001 #End to end uses learning rate of 0.01 for POS tagging and 0.015 for NER where lr is updated on each epoch with decay rate 0.05
#End to end also uses gradient clipping of 5.0
lr_decay = 0.0005 #GG uses 0.9; paper uses 0.05
nepochs = 75 #End to end paper saw best results at 50 epochs
batch_size = 10 #20 #End to end paper uses 10 #eval at 32

In [22]:
# TODO: make use of minibatches with fit_generator
# for i, (words, labels) in enumerate(minibatches(train, batch_size)):
words, labels = list(minibatches(train, len(train)))[0]  # NOTE: len(train) will return entire dataset!
#GG's version
char_ids, word_ids = zip(*words)
word_ids, sequence_lengths = pad_sequences(word_ids, pad_tok=9) #word_ids = vocab_chars?
char_ids, word_lengths = pad_sequences(char_ids, pad_tok=9, nlevels=2)
labels, _ = pad_sequences(labels, pad_tok=9)
#try with maxlengths
# char_ids, word_ids = zip(*words)
# word_ids, max_seq_length_b = pad_sequences(word_ids, pad_tok=0) #word_ids = vocab_chars?
# char_ids, max_word_length_b = pad_sequences(char_ids, pad_tok=0, nlevels=2)
# labels, _ = pad_sequences(labels, pad_tok=0)
#try with keras definition of pad_sequences
# char_ids, word_ids = zip(*words)
# word_ids = pad_sequences(word_ids, padding='post', value=0, maxlen=max_seq_length) #word_ids = vocab_chars?
# char_ids= pad_sequences(char_ids, padding='post', value=0)#, maxlen=max_word_length)
# labels = pad_sequences(labels, padding='post', value=0)

In [23]:
word_ids_array = np.array(word_ids)
word_ids_array.shape

(14041, 113)

In [24]:
# TODO: CHANGE PADDING VALUE FOR VAL & TEST

In [57]:
#validation/dev

words_dev, labels_dev = list(minibatches(dev, len(dev)))[0]  
char_ids_dev, word_ids_dev = zip(*words_dev)
word_ids_dev, sequence_lengths_dev = pad_sequences(word_ids_dev, 0)
char_ids_dev, word_lengths_dev = pad_sequences(char_ids_dev, pad_tok=0, nlevels=2)
labels_dev, _ = pad_sequences(labels_dev, 0)

In [26]:
#test

# words, labels = list(minibatches(test, len(test)))[0]  
# char_ids, word_ids = zip(*words)
# word_ids, sequence_lengths = pad_sequences(word_ids, 0)
# char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0, nlevels=2)
# labels, _ = pad_sequences(labels, 0)

In [27]:
# max_seq_length = #113 #max(sequence_lengths) None
# max_word_length = #24 #max(max(i) for i in word_lengths) #None

In [28]:
dropout = 0.5 # needs to be set before Dropout function- GG 0.5

In [29]:
# word_embeddings = word_ids, n_words, dim_word
# word_emb_input = Input((max_seq_length,))
word_emb_input = Input((None,))
mask_word = Masking(mask_value=9)(word_emb_input)
word_emb_output = Embedding(n_words, dim_word, weights=[embeddings], trainable=False)(mask_word)

In [30]:
# x = tf.plaaceholder(tf.float32, (None, None, None))
# y = tf.placeholder(tf.float32, (None))
# y_reshape = tf.reshape(y, (tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]))
# # y_reshape = tf.reshape(y, tf.shape(x)
# # y_reshape = tf.reshape(y, (10,  13, 7))
# sess = tf.keras.backend.get_session()
# xnp = np.random.rand(10, 13, 7)
# ynp = xnp.reshape((10*13*7,))
# print(xnp.shape)
# print(ynp.shape)
# print(sess.run(tf.shape(x), feed_dict={x: xnp, y: ynp}))
# y_reshape_np = sess.run(y_reshape, feed_dict={x: xnp, y: ynp})
# assert np.allclose(xnp, y_reshape_np)

In [31]:
#end to end paper claims to have applied dropout layer on character embeddings before inputting to a CNN in addition to before both layers of BLSTM
# char_emb_input = Input((max_seq_length, max_word_length)) 
char_emb_input = Input((None, None))
#comes in as sentences, words, characters and for the character part we want to just operate it over the character sentence by number of words and seq of characters so reshape so we have words by characters
char_emb_output = Lambda(lambda x: tf.keras.backend.reshape(x, (-1, tf.keras.backend.shape(x)[-1])))(char_emb_input)
mask_char = Masking(mask_value=9)(char_emb_output)  # TODO: make -1 a variable
char_emb_output = Embedding(n_char, dim_char)(mask_char) #need weights here?
# 2 sided LSTM below that we can change with forward and backward to see which is better performing
# char_emb_output = Bidirectional(LSTM(hidden_size_char, return_sequences=False))(char_emb_output)
char_emb_output = Dropout(dropout)(char_emb_output)
fw_LSTM = LSTM(hidden_size_char, return_sequences=False)(char_emb_output) #is this right?
bw_LSTM = LSTM(hidden_size_char, return_sequences=False, go_backwards=True)(char_emb_output)
char_emb_output = concatenate([fw_LSTM, bw_LSTM])
char_emb_output = Dropout(dropout)(char_emb_output)
char_emb_output = Lambda(lambda x, z: tf.keras.backend.reshape(x, (-1, tf.shape(z)[1], 2 * hidden_size_char)), arguments={"z": word_emb_input})(char_emb_output)

In [32]:
#concatenates word embedding and character embedding
x = concatenate([word_emb_output, char_emb_output])

In [33]:
x = Dropout(dropout)(x)
x = Bidirectional(LSTM(hidden_size_lstm, return_sequences=True))(x)  #should we turn this into two layers (fw and bw)?
# fw_LSTM_2 = LSTM(hidden_size_lstm, return_sequences=False)(x) #is this right?
# bw_LSTM_2 = LSTM(hidden_size_lstm, return_sequences=False, go_backwards=True)(x)
# x = concatenate([fw_LSTM_2, bw_LSTM_2])
x = Dropout(dropout)(x)
scores = Dense(n_tags)(x) 
softmax = Activation("softmax")(scores)
crf_layer = CRF(n_tags)
# crf = crf_layer(scores) #should we add this to attach to the softmax model? with SGD and gradiet clipping of 5.0?

In [34]:
model_softmax = Model([word_emb_input, char_emb_input], softmax) #should these be input

In [35]:
# model_softmax = Model(word_emb_input, softmax)

In [36]:
# model_crf = Model([word_emb_input, char_emb_input], crf) #should these be input

In [37]:
model_softmax.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None)   0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None)         0           input_2[0][0]                    
__________________________________________________________________________________________________
masking_2 (Masking)             (None, None)         0           lambda_1[0][0]                   
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    2800        masking_2[0][0]                  
__________________________________________________________________________________________________
dropout_1 

In [38]:
model_softmax.inputs

[<tf.Tensor 'input_1:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, ?, ?) dtype=float32>]

In [39]:
# model_softmax.predict([np.random.randn(10, 100), np.random.randn(10, 100, 12)])

In [40]:
# model_crf.summary()

In [41]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model_softmax).create(prog='dot', format='svg'))

In [42]:
adam_op = Adam(lr=lr, decay=lr_decay)

In [43]:
model_softmax.compile(loss="categorical_crossentropy", optimizer=adam_op, metrics=["accuracy"])

In [44]:
# model_crf.compile(loss=crf_layer.loss_function, optimizer=adam_op, metrics=["accuracy"])

In [45]:
char_ids_arr = np.array(char_ids)
word_ids_arr = np.array(word_ids)
labels_arr = np.array(labels)
# TODO: add one-hot label encoding to function down below
labels_arr_one_hot = np.eye(10)[labels] #10 if vocab_tags are different

In [ ]:
char_ids_arr_dev = np.array(char_ids_dev)
word_ids_arr_dev = np.array(word_ids_dev)
labels_arr_dev = np.array(labels_dev)
# TODO: add one-hot label encoding to function down below
labels_arr_one_hot_dev = np.eye(10)[labels_dev] #10 if vocab_tags are different

In [46]:
# char_ids_arr_dev = np.array(char_ids_dev)

In [47]:
# word_ids_arr_dev = np.array(word_ids_dev)

In [48]:
# labels_arr_dev = np.array(labels_dev)

In [49]:
# labels_arr_one_hot_dev = np.eye(9)[labels_dev]

In [50]:
# labels_arr_one_hot_dev.shape

In [51]:
# date = datetime.strftime(datetime.today(), "%y%m%d_%H%M%S")
# base_dir = f"models/{date}"
# if not os.path.exists(base_dir):
#     os.makedirs(base_dir)
# model_checkpoint = keras.callbacks.ModelCheckpoint(base_dir + "/{val_loss}_{epoch:03d}.hdf5")
# tb_callback = keras.callbacks.TensorBoard(log_dir='./logs')
# callbacks = [model_checkpoint, tb_callback]
# print(base_dir)

In [52]:
word_ids_arr.shape

(14041, 113)

In [53]:
char_ids_arr.shape

(14041, 113, 24)

In [54]:
labels_arr_one_hot.shape

(14041, 113, 10)

In [58]:
# Add callbacks:
# early stopping and saving best parameters
# learning rate decay
# tensorboard
# number of epochs without improving is 0 (for early stopping)
# could add gradient clipping (optional)
model_softmax.fit([word_ids_arr, char_ids_arr], labels_arr_one_hot, batch_size=batch_size, epochs=nepochs, validation_data=dev) 
#fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
#fit_generator(self, generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

# model_softmax.save(f"{base_dir}/train_softmax.hdf5") #final_softmax

ValueError: When passing validation_data, it must contain 2 (x_val, y_val) or 3 (x_val, y_val, val_sample_weights) items, however it contains 3250 items

In [ ]:
# model_softmax.save("softmax.hdf5")

In [ ]:
# model_softmax.save("softmax_with_val.hdf5")

In [ ]:
# model_softmax.save("softmax_with_masking_neg1.hdf5")

In [ ]:
# model_softmax.save("softmax_with_masking_nine.hdf5")

In [ ]:
model_softmax.save("softmax_test_5_9.hdf5")

In [ ]:
# model_crf.fit([word_ids_arr, char_ids_arr], labels_arr_one_hot, batch_size=batch_size, epochs=nepochs, validation_split=0.33)

In [ ]:
# model_crf.save("crf_with_val.hdf5")

In [ ]:
# base_dir  #models/180222_215523

In [ ]:
# lolz why so slow?
# model_softmax.load_weights("softmax_test2.hdf5") #100 epochs but symptoms of exploding gradients leading me to wanting to add gradient clipping
# model_softmax.load_weights("softmax_test.hdf5") #with 75 epochs on same tuning as test2
model_softmax.load_weights("softmax_test_5_9.hdf5") #with 75 epochs on same tuning as test2 with dev in training
# model_softmax.load_weights("softmax_with_masking_nine.hdf5")
# model_softmax.load_weights("softmax_with_masking_neg1.hdf5")
# model_softmax.load_weights(f"{base_dir}/train_softmax.hdf5")#"models/180222_215523/final_softmax.hdf5")#"0.11342436582348703_050.hdf5")

In [ ]:
#need matplotlib
# print(history.history.keys())

In [ ]:
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train'], loc='upper left')
# plt.show()

In [ ]:
# prediction_softmax = model_softmax.predict([word_ids_arr, char_ids_arr])

In [ ]:
#trying a test for masking


In [ ]:
# model_crf.load_weights("crf_with_val.hdf5")

In [ ]:
# prediction_crf = model_crf.predict([word_ids_arr, char_ids_arr])

In [ ]:
# print(prediction_crf)

In [ ]:
# evaluate the model
# scores_crf = model_crf.evaluate([word_ids_arr, char_ids_arr], labels_arr_one_hot) #x_test, y_test (when testing)
# print("%s: %.2f%%" % (model_crf.metrics_names[1], scores_crf[1]*100))

In [ ]:
# scores_softmax = model_softmax.evaluate([word_ids_arr, char_ids_arr], labels_arr_one_hot) #x_test, y_test (when testing)
# print("%s: %.2f%%" % (model_softmax.metrics_names[1], scores_softmax[1]*100))

In [ ]:
#train F1 evaluation

In [ ]:
#literally just copying data_utils.py

UNK = "$UNK$"
NUM = "$NUM$"
NONE = "O"

# special error message
class MyIOError(Exception):
    def __init__(self, filename):
        # custom error message
        message = """
ERROR: Unable to locate file {}.
FIX: Have you tried running python build_data.py first?
This will build vocab file from your train, test and dev sets and
trimm your word vectors.
""".format(filename)
        super(MyIOError, self).__init__(message)


class CoNLLDataset(object):
    """Class that iterates over CoNLL Dataset
    __iter__ method yields a tuple (words, tags)
        words: list of raw words
        tags: list of raw tags
    If processing_word and processing_tag are not None,
    optional preprocessing is appplied
    Example:
        ```python
        data = CoNLLDataset(filename)
        for sentence, tags in data:
            pass
        ```
    """
    def __init__(self, filename, processing_word=None, processing_tag=None,
                 max_iter=None):
        """
        Args:
            filename: path to the file
            processing_words: (optional) function that takes a word as input
            processing_tags: (optional) function that takes a tag as input
            max_iter: (optional) max number of sentences to yield
        """
        self.filename = filename
        self.processing_word = processing_word
        self.processing_tag = processing_tag
        self.max_iter = max_iter
        self.length = None


    def __iter__(self):
        niter = 0
        with open(self.filename) as f:
            words, tags = [], []
            for line in f:
                line = line.strip()
                if (len(line) == 0 or line.startswith("-DOCSTART-")):
                    if len(words) != 0:
                        niter += 1
                        if self.max_iter is not None and niter > self.max_iter:
                            break
                        yield words, tags
                        words, tags = [], []
                else:
                    ls = line.split(' ')
                    word, tag = ls[0],ls[-1]
                    if self.processing_word is not None:
                        word = self.processing_word(word)
                    if self.processing_tag is not None:
                        tag = self.processing_tag(tag)
                    words += [word]
                    tags += [tag]


    def __len__(self):
        """Iterates once over the corpus to set and store length"""
        if self.length is None:
            self.length = 0
            for _ in self:
                self.length += 1

        return self.length


def get_vocabs(datasets):
    """Build vocabulary from an iterable of datasets objects
    Args:
        datasets: a list of dataset objects
    Returns:
        a set of all the words in the dataset
    """
    print("Building vocab...")
    vocab_words = set()
    vocab_tags = set()
    for dataset in datasets:
        for words, tags in dataset:
            vocab_words.update(words)
            vocab_tags.update(tags)
    print("- done. {} tokens".format(len(vocab_words)))
    return vocab_words, vocab_tags


def get_char_vocab(dataset):
    """Build char vocabulary from an iterable of datasets objects
    Args:
        dataset: a iterator yielding tuples (sentence, tags)
    Returns:
        a set of all the characters in the dataset
    """
    vocab_char = set()
    for words, _ in dataset:
        for word in words:
            vocab_char.update(word)

    return vocab_char


def get_glove_vocab(filename):
    """Load vocab from file
    Args:
        filename: path to the glove vectors
    Returns:
        vocab: set() of strings
    """
    print("Building vocab...")
    vocab = set()
    with open(filename) as f:
        for line in f:
            word = line.strip().split(' ')[0]
            vocab.add(word)
    print("- done. {} tokens".format(len(vocab)))
    return vocab


def write_vocab(vocab, filename):
    """Writes a vocab to a file
    Writes one word per line.
    Args:
        vocab: iterable that yields word
        filename: path to vocab file
    Returns:
        write a word per line
    """
    print("Writing vocab...")
    with open(filename, "w") as f:
        for i, word in enumerate(vocab):
            if i != len(vocab) - 1:
                f.write("{}\n".format(word))
            else:
                f.write(word)
    print("- done. {} tokens".format(len(vocab)))


def load_vocab(filename):
    """Loads vocab from a file
    Args:
        filename: (string) the format of the file must be one word per line.
    Returns:
        d: dict[word] = index
    """
    try:
        d = dict()
        with open(filename) as f:
            for idx, word in enumerate(f):
                word = word.strip()
                d[word] = idx

    except IOError:
        raise MyIOError(filename)
    return d


def export_trimmed_glove_vectors(vocab, glove_filename, trimmed_filename, dim):
    """Saves glove vectors in numpy array
    Args:
        vocab: dictionary vocab[word] = index
        glove_filename: a path to a glove file
        trimmed_filename: a path where to store a matrix in npy
        dim: (int) dimension of embeddings
    """
    embeddings = np.zeros([len(vocab), dim])
    with open(glove_filename) as f:
        for line in f:
            line = line.strip().split(' ')
            word = line[0]
            embedding = [float(x) for x in line[1:]]
            if word in vocab:
                word_idx = vocab[word]
                embeddings[word_idx] = np.asarray(embedding)

    np.savez_compressed(trimmed_filename, embeddings=embeddings)


def get_trimmed_glove_vectors(filename):
    """
    Args:
        filename: path to the npz file
    Returns:
        matrix of embeddings (np array)
    """
    try:
        with np.load(filename) as data:
            return data["embeddings"]

    except IOError:
        raise MyIOError(filename)


def get_processing_word(vocab_words=None, vocab_chars=None,
                    lowercase=False, chars=False, allow_unk=True):
    """Return lambda function that transform a word (string) into list,
    or tuple of (list, id) of int corresponding to the ids of the word and
    its corresponding characters.
    Args:
        vocab: dict[word] = idx
    Returns:
        f("cat") = ([12, 4, 32], 12345)
                 = (list of char ids, word id)
    """
    def f(word):
        # 0. get chars of words
        if vocab_chars is not None and chars == True:
            char_ids = []
            for char in word:
                # ignore chars out of vocabulary
                if char in vocab_chars:
                    char_ids += [vocab_chars[char]]

        # 1. preprocess word
        if lowercase:
            word = word.lower()
        if word.isdigit():
            word = NUM

        # 2. get id of word
        if vocab_words is not None:
            if word in vocab_words:
                word = vocab_words[word]
            else:
                if allow_unk:
                    word = vocab_words[UNK]
                else:
                    raise Exception("Unknow key is not allowed. Check that "\
                                    "your vocab (tags?) is correct")

        # 3. return tuple char ids, word id
        if vocab_chars is not None and chars == True:
            return char_ids, word
        else:
            return word

    return f


def _pad_sequences(sequences, pad_tok, max_length):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
    Returns:
        a list of list where each sublist has same length
    """
    sequence_padded, sequence_length = [], []

    for seq in sequences:
        seq = list(seq)
        seq_ = seq[:max_length] + [pad_tok]*max(max_length - len(seq), 0)
        sequence_padded +=  [seq_]
        sequence_length += [min(len(seq), max_length)]

    return sequence_padded, sequence_length


def pad_sequences(sequences, pad_tok, nlevels=1):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
        nlevels: "depth" of padding, for the case where we have characters ids
    Returns:
        a list of list where each sublist has same length
    """
    if nlevels == 1:
        max_length = max(map(lambda x : len(x), sequences))
        sequence_padded, sequence_length = _pad_sequences(sequences,
                                            pad_tok, max_length)

    elif nlevels == 2:
        max_length_word = max([max(map(lambda x: len(x), seq))
                               for seq in sequences])
        sequence_padded, sequence_length = [], []
        for seq in sequences:
            # all words are same length now
            sp, sl = _pad_sequences(seq, pad_tok, max_length_word)
            sequence_padded += [sp]
            sequence_length += [sl]

        max_length_sentence = max(map(lambda x : len(x), sequences))
        sequence_padded, _ = _pad_sequences(sequence_padded,
                [pad_tok]*max_length_word, max_length_sentence)
        sequence_length, _ = _pad_sequences(sequence_length, 0,
                max_length_sentence)

    return sequence_padded, sequence_length


def minibatches(data, minibatch_size):
    """
    Args:
        data: generator of (sentence, tags) tuples
        minibatch_size: (int)
    Yields:
        list of tuples
    """
    x_batch, y_batch = [], []
    for (x, y) in data:
        if len(x_batch) == minibatch_size:
            yield x_batch, y_batch
            x_batch, y_batch = [], []

        if type(x[0]) == tuple:
            x = zip(*x)
        x_batch += [x]
        y_batch += [y]

    if len(x_batch) != 0:
        yield x_batch, y_batch


def get_chunk_type(tok, idx_to_tag):
    """
    Args:
        tok: id of token, ex 4
        idx_to_tag: dictionary {4: "B-PER", ...}
    Returns:
        tuple: "B", "PER"
    """
    tag_name = idx_to_tag[tok]
    print({"tag_name" : tag_name})
    tag_class = tag_name.split('-')[0]
    print({"tag_class" : tag_class})
    tag_type = tag_name.split('-')[-1]
    print({"tag_type" : tag_type})
    return tag_class, tag_type


def get_chunks(seq, tags):
    """Given a sequence of tags, group entities and their position
    Args:
        seq: [4, 4, 0, 0, ...] sequence of labels
        tags: dict["O"] = 4
    Returns:
        list of (chunk_type, chunk_start, chunk_end)
    Example:
        seq = [4, 5, 0, 3]
        tags = {"B-PER": 4, "I-PER": 5, "B-LOC": 3}
        result = [("PER", 0, 2), ("LOC", 3, 4)]
    """
    default = tags[NONE]
    idx_to_tag = {idx: tag for tag, idx in tags.items()}
    print({"idx_to_tag" : idx_to_tag})
    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        # End of a chunk 1
        if tok == default and chunk_type is not None:
            # Add a chunk.
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None
            print({"chunk": chunk})

        # End of a chunk + start of a chunk!
        elif tok != default:
            tok_chunk_class, tok_chunk_type = get_chunk_type(tok, idx_to_tag) #get_chunk_type used here
            print({"chunk_type" : tok_chunk_type})
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk_type, i
            elif tok_chunk_type != chunk_type or tok_chunk_class == "B":
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk_type, i
                print({"chunk_type" : chunk_type})
                print({"chunk_start" : chunk_start})
        else:
            pass

    # end condition
    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)
        print({"chunk" : chunk})

    return chunks

In [ ]:
#####TESTING CODE -- DELETE CELL WHEN FINISHED####

# def test_get_chunk_type:
#     """
#     Testing the function get_chunk_type to see if it will work with a non-tag number.
#     """
#     tok
# def get_chunk_type(tok, idx_to_tag):
#     """
#     Args:
#         tok: id of token, ex 4
#         idx_to_tag: dictionary {4: "B-PER", ...}
#     Returns:
#         tuple: "B", "PER"
#     """
#     tag_name = idx_to_tag[tok]
#     print({"tag_name" : tag_name})
#     tag_class = tag_name.split('-')[0]
#     print({"tag_class" : tag_class})
#     tag_type = tag_name.split('-')[-1]
#     print({"tag_type" : tag_type})
#     return tag_class, tag_type

seq_lens_arr = np.array(sequence_lengths)
# print(labels_arr[:seq_lens_arr])
labels_prob_arr = model_softmax.predict([word_ids_arr, char_ids_arr], batch_size) #shape(num sentences, max num words, num tags)
labels_pred_arr = np.argmax(labels_prob_arr, -1)
accs = []
correct_preds, total_correct, total_preds = 0., 0., 0.
for lab, lab_pred, seq_len in zip(labels_arr, labels_pred_arr, seq_lens_arr):
        # NOTE: labels & predictions are padded to the maximum number of words
        # in the batch.  Here, we use the actual sentence lengths to select out
        # the actual labels and corresponding predictions.
        lab = lab[:seq_len]
#         print(lab)
#         print(lab.shape)
        lab_pred = lab_pred[:seq_len]
        for n, i in enumerate(lab_pred):
                if i == 9:
                    lab_pred[n] = 0
#         print({"lab" : lab})
#         print({"lab type" : type(lab)})
#         print({"lab_pred type" : type(lab_pred)})
        y_ = lab
        y = lab_pred #mycnn(...) # for eg: [2, 2, 4]
        num_classes = 9 #change to 9 if can make length of seq_lens_arr

        confusion = tf.confusion_matrix(labels=y_, predictions=y, num_classes=num_classes)
        print(confusion)

#         tok =
#         idx_to_tag =
#         tag_name = idx_to_tag[tok]
#         print({"tag_name" : tag_name})
#         tag_class = tag_name.split('-')[0]
#         print({"tag_class" : tag_class})
#         tag_type = tag_name.split('-')[-1]
#         print({"tag_type" : tag_type})
#         return tag_class, tag_type

# def get_chunks(seq, tags):
        """Given a sequence of tags, group entities and their position
        Args:
            seq: [4, 4, 0, 0, ...] sequence of labels
            tags: dict["O"] = 4
        Returns:
            list of (chunk_type, chunk_start, chunk_end)
        Example:
            seq = [4, 5, 0, 3]
            tags = {"B-PER": 4, "I-PER": 5, "B-LOC": 3}
            result = [("PER", 0, 2), ("LOC", 3, 4)]
        """
#         seq = lab
#         tags = vocab_tags
#         default = tags[NONE]
#         idx_to_tag = {idx: tag for tag, idx in tags.items()}
#         print({"idx_to_tag" : idx_to_tag})
#         chunks = []
#         chunk_type, chunk_start = None, None
#         for i, tok in enumerate(seq):
#             # End of a chunk 1
#             if tok == default and chunk_type is not None:
#                 # Add a chunk.
#                 chunk = (chunk_type, chunk_start, i)
#                 chunks.append(chunk)
#                 chunk_type, chunk_start = None, None
#                 print({"chunk": chunk})

#             # End of a chunk + start of a chunk!
#             elif tok != default:
#                 tok_chunk_class, tok_chunk_type = get_chunk_type(tok, idx_to_tag) #get_chunk_type used here
#                 print({"chunk_type" : tok_chunk_type})
#                 if chunk_type is None:
#                     chunk_type, chunk_start = tok_chunk_type, i
#                 elif tok_chunk_type != chunk_type or tok_chunk_class == "B":
#                     chunk = (chunk_type, chunk_start, i)
#                     chunks.append(chunk)
#                     chunk_type, chunk_start = tok_chunk_type, i
#                     print({"chunk_type" : chunk_type})
#                     print({"chunk_start" : chunk_start})
# #                 print({"chunk": chunk})
# #                 print({"chunk_type" : tok_chunk_type})
# #                 print({"chunk_type" : chunk_type})
# #                 print({"chunk_start" : chunk_start})
                
#             else:
#                 pass
            
            
#         # end condition
#         if chunk_type is not None:
#             chunk = (chunk_type, chunk_start, len(seq))
#             chunks.append(chunk)
#             print({"chunk" : chunk})
# #         return chunks

words, labels = list(minibatches(train, len(train)))[0]  # NOTE: len(train) will return entire dataset!
#GG's version
char_ids, word_ids = zip(*words)
word_ids, sequence_lengths = pad_sequences(word_ids, pad_tok=9) #word_ids = vocab_chars?
char_ids, word_lengths = pad_sequences(char_ids, pad_tok=9, nlevels=2)
labels, _ = pad_sequences(labels, pad_tok=9)

In [ ]:
def extract_data(dataset):
    """Extract words and labels from a dataset.
    
    Args:
      dataset: A CoNLL dataset.
    
    Returns:
      Word ids, char ids, and labels, from a CoNLL dataset,
      all as NumPy arrays.
    """
    
    words, labels = list(minibatches(dataset, len(dataset)))[0]  # NOTE: len(train) will return entire dataset!
    char_ids, word_ids = zip(*words)
    # TODO: we need to be able to pad sequences to a specific length
    # or the training vs. val vs. test datasets will likely have
    # different max lengths not equal to 113
    
    word_ids, sequence_lengths = pad_sequences(word_ids, pad_tok=9)
    char_ids, word_lengths = pad_sequences(char_ids, pad_tok=9, nlevels=2)
    labels, _ = pad_sequences(labels, pad_tok=9)

#     word_ids, max_seq_length_b = pad_sequences(word_ids, pad_tok=0)
#     char_ids, max_word_length_b = pad_sequences(char_ids, pad_tok=0, nlevels=2)
#     labels, _ = pad_sequences(labels, pad_tok=0)

#     word_ids, sequence_lengths = pad_sequences(word_ids, padding='post', value=0) #word_ids = vocab_chars?
#     char_ids, word_lengths = pad_sequences(char_ids, padding='post', value=0, nlevels=2)
#     labels, _ = pad_sequences(labels, padding='post', value=0)
    word_ids_arr = np.array(word_ids)
    char_ids_arr = np.array(char_ids)
    labels_arr = np.array(labels)
    # TODO: add one-hot encoding of labels
    seq_lens_arr = np.array(sequence_lengths)
    return word_ids_arr, char_ids_arr, labels_arr, seq_lens_arr


def predict_labels(model, word_ids_arr, char_ids_arr, seq_lens_arr, batch_size=32):
    """Predict labels for a set of words.
    
    Args:
      model: A Keras Model that accepts char ids and word ids
        and returns label probs.
      word_ids_arr: A NumPy array of word ids for sentences of shape
        (num sentences, max num words).
      char_ids_arr: A NumPy array of char ids for sentences of shape
        (num sentences, max num words, max num chars).
      seq_lens_arr: A NumPy array of sentence lengths, of
        shape (num sentences, actual num words). 
    
    Returns:
      A NumPy array of shape (num sentences, num words)
      containing the predicted tags for each word.
    """
#     model.load_weights("softmax_with_masking_nine.hdf5")
    labels_prob_arr = model.predict([word_ids_arr, char_ids_arr], batch_size) #shape(num sentences, max num words, num tags)
#     labels_prob_arr = model.predict(word_ids_arr, batch_size) #shape(num sentences, max num words, num tags) #DELETE
    labels_pred_arr = np.argmax(labels_prob_arr, -1) 
    return labels_pred_arr


def compute_metrics(labels_arr, labels_pred_arr, seq_lens_arr, vocab_tags): #commented out to play with it below but this is the og
    """Compute accuracy and F1.
    
    Args:
      labels_arr: A NumPy array of correct tags of shape
        (num sentences, max num words).
      labels_pred_arr: A NumPy array of predicted tags of
        shape (num sentences, max num words).
      seq_lens_arr: A NumPy array of sentence lengths, of
        shape (num sentences, actual num words).
      vocab_tags: Dictionary of tag strings to tag numbers.
      
    Returns:
      Dictionary with accuracy `acc` and F1 score `f1`.
    """
    accs = []
    correct_preds, total_correct, total_preds = 0., 0., 0.
#     import pdb; pdb.set_trace()

#     def get_chunk_type(tok, idx_to_tag):
#     """
#     Args:
#         tok: id of token, ex 4
#         idx_to_tag: dictionary {4: "B-PER", ...}
#     Returns:
#         tuple: "B", "PER"
#     """
#     tag_name = idx_to_tag[tok]
#     print(tag_name)
#     tag_class = tag_name.split('-')[0]
#     tag_type = tag_name.split('-')[-1]
#     return tag_class, tag_type
    for lab, lab_pred, seq_len in zip(labels_arr, labels_pred_arr, seq_lens_arr):
        # NOTE: labels & predictions are padded to the maximum number of words
        # in the batch.  Here, we use the actual sentence lengths to select out
        # the actual labels and corresponding predictions.
        lab = lab[:seq_len]
        lab_pred = lab_pred[:seq_len]
        for n, i in enumerate(lab_pred):
            if i == 9:
                lab_pred[n] = 0
        
        accs += [a==b for (a, b) in zip(lab, lab_pred)]
        
        # chunk up the labels & predictions by entities (i.e., if there
        # are multi-word entities, group those together), and store each
        # entity in a tuple
        lab_chunks      = set(get_chunks(lab, vocab_tags))
#         print({"lab_chunks": lab_chunks})
#         lab_chunk_type = set(get_chunk_type(lab, vocab_tags))
#         print({"lab_chunk_type": lab_chunk_type})
        lab_pred_chunks = set(get_chunks(lab_pred, vocab_tags))
#         print({"lab_pred_chunks": lab_pred_chunks})
#         lab_pred_chunk_type = set(get_chunk_type(lab_pred, vocab_tags))
#         print({"lab_pred_chunk_type": lab_pred_chunk_type})

        correct_preds += len(lab_chunks & lab_pred_chunks)
        total_preds   += len(lab_pred_chunks)
        total_correct += len(lab_chunks)
        
#         idx_to_tag = {idx: tag for tag, idx in tags.items()}
#         print(idx_to_tag)
        
    p   = correct_preds / total_preds if total_preds > 0 else 0 
    r   = correct_preds / total_correct if total_correct > 0 else 0
    f1  = 2 * p * r / (p + r) if correct_preds > 0 else 0
    acc = np.mean(accs)
#     y_ = lab
#     y = lab_pred #mycnn(...) # for eg: [2, 2, 4]
#     num_classes = 9 #change to 9 if can make length of seq_lens_arr

#     confusion = tf.confusion_matrix(labels=y_, predictions=y, num_classes=num_classes)
#     print({"confusion" : confusion})
    print ({"precision": p})
    print ({"recall": r})
    print ({"total_correct": total_correct})
    return {"acc": 100*acc, "f1": 100*f1}

In [ ]:
# if in file, could run with `py.test file.py`
def test_compute_metrics():
    vocab_tags = {
        'O': 0,
        'B-PER': 1,
        'I-PER': 2,
        'B-MISC': 3,
        'I-MISC': 4,
        'B-ORG': 5,
        'I-ORG': 6,
        'B-LOC': 7,
        'I-LOC': 8
    }
#     vocab_tags = {
#             'O': 1,
#             'B-PER': 2,
#             'I-PER': 3,
#             'B-MISC': 4,
#             'I-MISC': 5,
#             'B-ORG': 6,
#             'I-ORG': 7,
#             'B-LOC': 8,
#             'I-LOC': 9
#         }
    
    def test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn):
        metrics = compute_metrics(labels, labels_pred, seq_lens, vocab_tags)
        p = tp / (tp + fp)
        r = tp / (tp + fn)
        f1 = 2*p*r / (p+r) * 100
        assert np.allclose(metrics['acc'], acc), f"{metrics['acc']}=/={acc}"
        assert np.allclose(metrics['f1'], f1), f"{metrics['f1']}=/={f1}"
    
    # all beginning or null tags
    labels = np.array(     [[7, 3, 3], [5, 1, 1, 0]])
    labels_pred = np.array([[0, 0, 3], [7, 1, 1, 0]])
    seq_lens = np.array([3, 4])
    acc = 4/7 * 100
    tp=3
    fp=1
    fn=3
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)
    
    # all inside or null tags
    labels = np.array(     [[8, 4, 0], [2, 6, 8, 6]])
    labels_pred = np.array([[6, 4, 2], [8, 4, 8, 6]])
    seq_lens = np.array([3, 4])
    acc = 3/7 * 100
    tp=3
    fp=4
    fn=3
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)
    
    # all inside or null tags, with multiple insides in a row making a single entity
    labels = np.array(     [[8, 4, 0], [2, 2, 8, 6]])
    labels_pred = np.array([[6, 4, 2], [8, 4, 8, 6]])
    seq_lens = np.array([3, 4])
    acc = 3/7 * 100
    tp=3
    fp=4
    fn=2
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)

    # mix of beginning, inside, and null tags, but no multi-word entities
    labels = np.array(     [[8, 6, 0], [4, 3, 5, 2]])
    labels_pred = np.array([[1, 6, 0], [5, 3, 5, 2]])
    seq_lens = np.array([3, 4])
    acc = 5/7 * 100
    tp=4
    fp=2
    fn=2
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)
    
    # another mix of beginning, inside, and null tags, but no multi-word entities: got rid of 0s and 2s
    labels = np.array(     [[8, 6, 4], [4, 3, 5, 0]])
    labels_pred = np.array([[1, 6, 4], [5, 3, 5, 3]])
    seq_lens = np.array([3, 4])
    acc = 4/7 * 100
    tp=4
    fp=3
    fn=2
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)
    
    # another mix of beginning, inside, and null tags, but no multi-word entities: exactly the same
    labels = np.array(     [[4, 7, 4], [8, 8, 8, 8]]) #mismatched 2s and 0s result in error
    labels_pred = np.array([[4, 7, 4], [8, 8, 8, 8]]) #adding matched 2s result in error
    seq_lens = np.array([3, 4])
    metrics = compute_metrics(labels, labels_pred, seq_lens, vocab_tags)
    acc = 7/7 * 100
    tp=7
    fp=0
    fn=0
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)
    
    # mix of beginning, inside, and null tags, with multi-word entities
    labels = np.array(     [[7, 0, 0], [3, 1, 2, 3]])
    labels_pred = np.array([[7, 3, 0], [2, 1, 2, 4]])
    seq_lens = np.array([3, 4])
    acc = 4/7 * 100
    tp=3  #2
    fp=2  #3
    fn=1  #2
#     import pdb; pdb.set_trace()
    # THIS APPEARS TO BE A BUG WITH `get_chunks`
    # if we look at the second sentence, I printed out the chunked values:
    #  ```(Pdb) pp lab
    #  [3, 1, 2, 3]
    #  (Pdb) pp lab_pred
    #  [2, 1, 2, 4]
    #  (Pdb) n
    #  > <ipython-input-178-11c279c3838c>(76)compute_metrics()
    #  -> lab_chunks = set(get_chunks(lab, vocab_tags))
    #  (Pdb) n
    #  > <ipython-input-178-11c279c3838c>(77)compute_metrics()
    #  -> lab_pred_chunks = set(get_chunks(lab_pred, vocab_tags))
    #  (Pdb) n
    #  > <ipython-input-178-11c279c3838c>(79)compute_metrics()
    #  -> correct_preds += len(lab_chunks & lab_pred_chunks)
    #  (Pdb) pp lab_chunks
    #  {('MISC', 0, 1), ('PER', 1, 3), ('MISC', 3, 4)}
    #  (Pdb) pp lab_pred_chunks
    #  {('PER', 0, 1), ('PER', 1, 3), ('MISC', 3, 4)}
    #  (Pdb) pp lab_chunks & lab_pred_chunks
    #  {('MISC', 3, 4), ('PER', 1, 3)}
    #  ```
    # the first words are different (3=MISC vs 2=PER)
    # the 2nd & 3rd words form one entity (1=B-PER, 2=I-PER)
    # but look at the last words
    # 3 = B-MISC, while 4 = I-MISC
    # so they should be different
    # but the chunking just outputs ('MISC', 3, 4) for both (edited)
    # and so it ends up counting that as correct
    #
    # THE COMMENTED OUT TP/FP/FN VALUES ARE THE ONES THAT SHOULD BE CORRECT,
    # BECAUSE THE FINAL WORD IN THE SECOND SENTENCE SHOULD NOT BE CORRECT
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)
    
    # mix of beginning, inside, and null tags, with multi-word entities
    # and multiple inside tags
    labels = np.array(     [[7, 0, 0], [3, 1, 2, 3]])
    labels_pred = np.array([[3, 2, 2], [2, 1, 2, 4]])
    seq_lens = np.array([3, 4])
    acc = 2/7 * 100
    # SAME ISSUE AS THE PREVIOUS TEST
    # THE COMMENTED OUT VALUES ARE THE ONES THAT SHOULD BE CORRECT,
    # BECAUSE THE FINAL WORD IN THE SECOND SENTENCE SHOULD NOT BE CORRECT
    tp=2  #1
    fp=3  #4
    fn=2  #3
#     import pdb; pdb.set_trace()
    test_scenario(labels, labels_pred, seq_lens, acc, tp, fp, fn)

In [ ]:
# vocab_tags = {
#     'O': 0,
#     'B-PER': 1,
#     'I-PER': 2,
#     'B-MISC': 3,
#     'I-MISC': 4,
#     'B-ORG': 5,
#     'I-ORG': 6,
#     'B-LOC': 7,
#     'I-LOC': 8
# }

In [ ]:
# test_compute_metrics()

In [ ]:
word_ids_arr, char_ids_arr, labels_arr, seq_lens_arr = extract_data(train) 
labels_pred_arr = predict_labels(model_softmax, word_ids_arr, char_ids_arr, seq_lens_arr)
metrics = compute_metrics(labels_arr, labels_pred_arr, seq_lens_arr, vocab_tags)
print(metrics)

In [ ]:
word_ids_arr, char_ids_arr, labels_arr, seq_lens_arr = extract_data(dev) 
labels_pred_arr = predict_labels(model_softmax, word_ids_arr, char_ids_arr, seq_lens_arr)
metrics = compute_metrics(labels_arr, labels_pred_arr, seq_lens_arr, vocab_tags)
print(metrics)

In [ ]:
for a in [word_ids_arr, char_ids_arr, labels_arr, seq_lens_arr]:
    print(a.shape)
    

In [ ]:
dev

In [ ]:
len(test)

In [ ]:
len(train)

In [ ]:
len(dev)
dev_array = np.array(dev, dtype=float)

In [ ]:
labels_prob_arr = model_softmax.predict([word_ids_arr, char_ids_arr], batch_size) #shape(num sentences, max num words, num tags)
labels_pred_arr = np.argmax(labels_prob_arr, -1)

In [ ]:
labels_pred_arr.shape

In [ ]:
seq_lens_arr = np.array(sequence_lengths)
y_ = labels_arr
y = labels_pred_arr #mycnn(...) # for eg: [2, 2, 4]
num_classes = 9 #change to 9 if can make length of seq_lens_arr

confusion = tf.confusion_matrix(labels=y_, predictions=y, num_classes=num_classes)

In [ ]:
print(labels_prob_arr[0])
print(labels_pred_arr[0])

In [ ]:
seq_lens_arr

In [ ]:
print(labels_arr[0][:seq_lens_arr[0]])
print(labels_prob_arr[0].shape)
print(labels_prob_arr[0][:seq_lens_arr[0]])
print(np.argmax(labels_prob_arr[0][:seq_lens_arr[0]], axis=-1))
# print(np.argmax(labels_prob_arr[0][:seq_lens_arr[0]]))
print(labels_pred_arr[0])

In [ ]:
print(labels_arr[0])
print(labels_arr[0][:seq_lens_arr[0]])
print(labels_pred_arr[0])
print(labels_pred_arr[0][:seq_lens_arr[0]])

In [ ]:
print(labels_arr[1])
print(labels_arr[1][:seq_lens_arr[1]])
print(labels_pred_arr[1])
print(labels_pred_arr[1][:seq_lens_arr[1]])

In [ ]:
print(labels_arr[10])
print(labels_arr[10][:seq_lens_arr[10]])
print(labels_pred_arr[10])
print(labels_pred_arr[10][:seq_lens_arr[10]])

In [ ]:
for lab, lab_pred, seq_len in zip (labels_arr, labels_pred_arr, seq_lens_arr):
        # NOTE: labels & predictions are padded to the maximum number of words
        # in the batch.  Here, we use the actual sentence lengths to select out
        # the actual labels and corresponding predictions.
    lab = lab[:seq_len]
    lab_pred = lab_pred[:seq_len]
    for n, i in enumerate(lab_pred):
        if i == 0:
            lab_pred[n] = 1

    print(len(lab_pred))
    print(len(lab))

---

In [ ]:
#predictor only seems to predict 'O' and 'B-PER' tags
# labels_pred_seq_lens_arr = (labels_pred_arr[:seq_lens_arr])
unique, counts = np.unique(labels_pred_arr, return_counts=True) #labels are a list not a numpy array
dict(zip(unique, counts))

In [ ]:
unique, counts = np.unique(labels_arr, return_counts=True)
dict(zip(unique, counts))
# type(labels_arr)

In [ ]:
#difference in "0s"
print("number of incorrect 0 tags: " )
1391215 - 1383012

In [ ]:
print("number of incorrect 2 tags: " )
195418- 169578

In [ ]:
#checking total number of incorrect 0 and 2 tags from predicted and comparing to the 1, 3-8 tags of the labels
print((8203+25840), (11128+10001+37+8286+4556+24+11))

In [ ]:
1391215+195418 #total num of pred
11128+10001+37+8286+4556+24+11+1383012+169578 # total num of labels (they are the same)

In [ ]:
1552590/1586633 #num correct predicted out of total predicted which is the same as total lables (not looking for correct placement just correct number of labels each)
#same accuracy as keras evaluation
#need to consider placement & chunking

In [ ]:
print(labels_pred)

In [ ]:
print(labels)

In [ ]:
labels_pred.shape

In [ ]:
labels_arr.shape

In [ ]:
print(labels_arr)

In [ ]:
correct_placement = np.equal(labels_pred, labels_arr)
unique, counts = np.unique(correct_placement, return_counts=True) #labels are a list not a numpy array
dict(zip(unique, counts))

In [ ]:
#accuracy considering placement is the same as keras evaluation
148173/1586633

In [ ]:
# words, labels = extract_data(train)
# labels_pred = predict_labels(words, model_crf)
# metrics = compute_metrics(labels, labels_pred, vocab_tags)
# print(metrics)

In [ ]:
# run_evaluate(train, model_softmax)

In [ ]:
 def run_evaluate(self, test):
        """Evaluates performance on test set
        Args:
            test: dataset that yields tuple of (sentences, tags)
        Returns:
            metrics: (dict) metrics["acc"] = 98.4, ...
        """
        accs = []
        correct_preds, total_correct, total_preds = 0., 0., 0.
        for words, labels in minibatches(test, self.config.batch_size):
            labels_pred, sequence_lengths = self.predict_batch(words)

            for lab, lab_pred, length in zip(labels, labels_pred,
                                             sequence_lengths):
                lab      = lab[:length]
                lab_pred = lab_pred[:length]
                accs    += [a==b for (a, b) in zip(lab, lab_pred)]

                lab_chunks      = set(get_chunks(lab, self.config.vocab_tags))
                lab_pred_chunks = set(get_chunks(lab_pred,
                                                 self.config.vocab_tags))

                correct_preds += len(lab_chunks & lab_pred_chunks)
                total_preds   += len(lab_pred_chunks)
                total_correct += len(lab_chunks)

        p   = correct_preds / total_preds if correct_preds > 0 else 0
        r   = correct_preds / total_correct if correct_preds > 0 else 0
        f1  = 2 * p * r / (p + r) if correct_preds > 0 else 0
        acc = np.mean(accs)

        return {"acc": 100*acc, "f1": 100*f1}

In [ ]:
run_evaluate(prediction_softmax, test)

In [ ]:
run_evaluate(prediction_softmax, test)

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(y_test, y_score) #what are these values?

# TODO:
- add CRF training (layer + loss)
- add different metric
- check that training yields same results
- should bidirectional lstm use separate forward/backward cells?
- clean up notebook
- add prediction
- check for anything else that's missing
- replace code with keras stuff from notebook

In [ ]:
len(word_ids)

In [ ]:
for i, (dev_words, dev_labels) in enumerate(minibatches(dev, batch_size)): #len(dev))):
    dev_char_ids, dev_word_ids = zip(*dev_words)
    dev_word_ids, dev_sequence_lengths = pad_sequences(dev_word_ids,0)
    dev_char_ids, dev_word_lengths = pad_sequences(dev_char_ids, pad_tok = 0, nlevels = 2)
    dev_labels, _ = pad_sequences(labels, 0)

In [ ]:
len(dev)

In [ ]:
dev_char_ids_arr = np.array(dev_char_ids)

In [ ]:
dev_char_ids_reshape = np.reshape(dev_char_ids_arr, (dev_char_ids_arr.shape[0],dev_char_ids_arr.shape[1] * dev_char_ids_arr.shape[2]))

In [ ]:
scores = model.evaluate([np.array(dev_word_ids), dev_char_ids_reshape], np.array(dev_labels), verbose = 1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
test_text = "Bob works at IBM in France"

In [ ]:
embeddings

In [ ]:
tokenizer.texts_to_sequences(test_text)

In [ ]:
prediction = model.predict(np.array(tokenizer.texts_to_sequences(test_text)))
print(prediction)

In [ ]:
processing_word = get_processing_word(vocab_words, vocab_chars, lowercase=True, chars=True)

In [ ]:
words = [processing_word(w) for w in test_text.split(" ")]

In [ ]:
test_text.split(" ")

In [ ]:
c_ids_arr = []
w_ids_arr = []

In [ ]:
# word_id or word_ids?
for (char_ids, word_id) in words:
    c_ids_arr.append(char_ids)
    print(word_id)
    w_ids_arr.append(word_id)

In [ ]:
c_ids_arr

In [ ]:
w_ids_arr

In [ ]:
pred_char_ids = pad_sequences([c_ids_arr], pad_tok=0, nlevels=2)

In [ ]:
pred_word_ids = pad_sequences([w_ids_arr], pad_tok=0)

In [ ]:
prediction = model.predict([np.array(pred_char_ids), np.array(pred_word_ids)])
print(prediction)

In [ ]:
if type(words[0]) == tuple:
    words = zip(*words)

In [ ]:
np.array(word_ids).shape

In [ ]:
len(words)

In [ ]:
prediction = model.predict([words])
print(prediction)